<a href="https://colab.research.google.com/github/wandb/edu/blob/main/decision-opt-course/3_dynamic_decision_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{decisionopt-nb3b} -->

# Lesson 3 - Dynamic Decision Optimization

In [ ]:
import os 
import pandas as pd
import pickle
import torch
import wandb

os.environ["WANDB_QUIET"] = "true"
project_name = "Dynamic Inventory Management for Bimbo"
decision_data = pd.read_parquet('decision_data.parquet')
decision_data.head()

In [ ]:
sample_store_and_product = decision_data.query(
"Agencia_ID == 1110 & Canal_ID == 7 & Ruta_SAK == 3301 & Cliente_ID == 15766 & Producto_ID == 1238"
)

In [ ]:
store_product_ids = [
    "Agencia_ID",
    "Canal_ID",
    "Ruta_SAK",
    "Cliente_ID",
    "Producto_ID",
]

numerical_cols = [
    "Venta_uni_hoy",
    "Venta_hoy",
]

model = torch.load("predictive_model.pt")
with open('catgeorical_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

categorical_for_prediction = sample_store_and_product[store_product_ids].values
categorical_encoded = encoder.transform(categorical_for_prediction)
categorical_tensor = torch.from_numpy(categorical_encoded).long()
categorical_tensor = [categorical_tensor[:, i] for i in range(categorical_tensor.shape[1])]

numerical_tensor = torch.from_numpy(sample_store_and_product[numerical_cols].values).float()
model.eval()
with torch.no_grad():
    prediction = model(categorical_tensor, numerical_tensor)
prediction

In [ ]:
sample_store_and_product = sample_store_and_product.assign(predicted_demand = prediction.numpy())
sample_store_and_product

In [ ]:
def add_preds_to_df(df):
    categorical_for_prediction = df[store_product_ids].values
    categorical_encoded = encoder.transform(categorical_for_prediction)
    categorical_tensor = torch.from_numpy(categorical_encoded).long()
    categorical_tensor = [categorical_tensor[:, i] for i in range(categorical_tensor.shape[1])]
    numerical_tensor = torch.from_numpy(df[numerical_cols].values).float()
    model.eval()
    with torch.no_grad():
        prediction = model(categorical_tensor, numerical_tensor)
    return df.assign(predicted_demand = prediction.numpy())

sample_store_and_product = decision_data.query(
"Agencia_ID == 1110 & Canal_ID == 7 & Ruta_SAK == 3301 & Cliente_ID == 15766 & Producto_ID == 1238"
)
add_preds_to_df(sample_store_and_product)

In [ ]:
def add_col_with_initial_value(df, col_name, value):
    df.loc[df.index[0], col_name] = value
    return df

def simulate_outcomes(df, decision_rule):
    df = df.copy()
    df = add_preds_to_df(df)
    df = add_col_with_initial_value(df, "old_stock", 0)
    first_stocking_decision = decision_rule(df.iloc[0])
    df = add_col_with_initial_value(df, "new_stock", first_stocking_decision)
    first_shortage = max(0, df.iloc[0].predicted_demand - df.iloc[0].new_stock)
    first_amount_sold = min(df.iloc[0].Demanda_uni_equil, df.iloc[0].new_stock + df.iloc[0].old_stock)
    df = add_col_with_initial_value(df, "shortage", first_shortage)
    df = add_col_with_initial_value(df, "total_sold", first_amount_sold)

    # Sometimes can use .shift pattern
    prev_period = df.iloc[0, :]
    for i in df.index[1:]:
        df.loc[i, "old_stock"] = max(0,
                                    min(prev_period.old_stock + prev_period.new_stock - prev_period.Demanda_uni_equil,
                                     prev_period.new_stock
                                    ))
        df.loc[i, "new_stock"] = decision_rule(df.loc[i])
        stock_on_hand = df.loc[i, "old_stock"] + df.loc[i, "new_stock"]
        df.loc[i, "shortage"] = max(0, df.loc[i, "Demanda_uni_equil"] - stock_on_hand)
        df.loc[i, "total_sold"] = min(df.loc[i, "Demanda_uni_equil"], stock_on_hand)
        df.loc[i, "spoilage"] = max(0, df.loc[i, "old_stock"] - df.loc[i, "Demanda_uni_equil"])
        prev_period = df.loc[i]
    return df

def first_decision_rule(state):
    return 1

simulate_outcomes(sample_store_and_product, first_decision_rule)

## Scaling to multiple stores

In [ ]:
decision_data.groupby('Agencia_ID').size()

In [ ]:
decision_validation_data = decision_data.query('Agencia_ID == 1110')
decision_holdout_data = decision_data.query('Agencia_ID == 24049')

In [ ]:
def objective_function(df):
    return df.total_sold.sum() - 3*df.shortage.sum() - 0.5 * df.spoilage.sum() - 0.5*df.old_stock.sum()

def log_metrics(outcomes, decision_function, tags=None):
    with wandb.init(project=project_name,
                    name=decision_function.__name__,
                    job_type="simulation outcomes",
                    tags=tags
                    ):
        wandb.log({
            "number_of_orders": outcomes.new_stock.count(),
            "total_inventory_orders": outcomes.new_stock.sum(),
            "number_of_shortages": (outcomes.shortage > 0).sum(),
            "total_shortage": outcomes.shortage.sum(),
            "total_sold": outcomes.total_sold.sum(),
            "total_old_stock": outcomes.old_stock.sum(),
            "full_outcome": outcomes[store_product_ids + ['Semana', 'old_stock', 'new_stock', 'shortage', 'total_sold', 'spoilage']],
            "objective_function": objective_function(outcomes)
        })
    return

def simulate_multiple_stores_and_products(raw_data, decision_function, tags, log=True):
    groups = raw_data.groupby(store_product_ids)
    outcomes = pd.concat([simulate_outcomes(group, decision_function) for _, group in groups])
    if log:
        log_metrics(outcomes, decision_function, tags)
    return outcomes

simulate_multiple_stores_and_products(decision_validation_data, first_decision_rule, tags=["agencia_1110"])

In [ ]:
import numpy as np

def predicted_need(state):
    return np.ceil(state.predicted_demand - state.old_stock)

def predicted_need_plus_one(state):
    return predicted_need(state) + 1

def predicted_demand(state):
    return np.ceil(state.predicted_demand)

for rule in [first_decision_rule, predicted_need, predicted_need_plus_one, predicted_demand]:
    simulate_multiple_stores_and_products(decision_validation_data, rule, tags=["agencia_1110"])


# Programmatic Optimization

In [ ]:
def linear_decision_function_factory(constant, predicted_demand_mult, old_stock_mult):
    def decision_function(state):
        return constant + predicted_demand_mult * state.predicted_demand + old_stock_mult * state.old_stock
    return decision_function

def objective(params):
    decision_function = linear_decision_function_factory(params.constant, params.predicted_demand_mult, params.old_stock_mult)
    outcomes = simulate_multiple_stores_and_products(decision_validation_data, decision_function, tags=["agencia_1110"], log=False)
    return objective_function(outcomes)

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'objective_function',
        'goal': 'maximize'
    },
    'parameters': {
        'constant': {
            'distribution': 'uniform',
            'min': 0,
            'max': 5
        },
        'predicted_demand_mult': {
            'distribution': 'uniform',
            'min': 0,
            'max': 1.5
        },
        'old_stock_mult': {
            'distribution': 'uniform',
            'min': -1.5,
            'max': 0,
        }
    }
}

def main():
    wandb.init(project=project_name)
    score = objective(wandb.config)
    wandb.log({'objective_function': score})

sweep_id = wandb.sweep(sweep_config, project=project_name)
wandb.agent(sweep_id, main, count=20)